In [1]:
import os
#os.chdir('..')
input_folder = "Input"
output_folder = "Output"
basepath = os.getcwd()
input_path = os.path.join(basepath, input_folder)
output_path = os.path.join(basepath, output_folder)
image_input = input_path + '\\'
image_output = output_path + '\\'
print(basepath)
print(input_path)

C:\Users\chunk\Jupyter Projects\colorizeBlackandWhite
C:\Users\chunk\Jupyter Projects\colorizeBlackandWhite\Input


In [2]:
os.chdir(basepath)
os.chdir('DeOldify')

In [3]:
#HIDE
import io
import PIL
from PIL import Image
from deoldify import device
from deoldify.device_id import DeviceId
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
import torch
import shutil

device.set(device=DeviceId.GPU0)
if not torch.cuda.is_available():
    print('GPU not available.')

In [4]:
#HIDE

import fastai
from deoldify.visualize import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

In [5]:
#HIDE

colorizer = get_image_colorizer(artistic=True)

In [6]:
#HIDE
grand = widgets.IntSlider(min=15,max=39)

url_pic = widgets.Text(placeholder='Please put url here: ')

file = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


button = widgets.Button(description='Colorize',
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
    )


uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)


restoreBox = widgets.Checkbox(
    value=False,
    description='Restore',
    disabled=False,
    indent=False
)

def clear_output_folder():
    os.chdir(output_path)
    file_list = os.listdir()
    for image in file_list:
        try:
            os.remove(image)
        except:
            os.rmdir(image)
    os.chdir(basepath+'\\DeOldify\\result_images')
    file_list = os.listdir()
    for image in file_list:
        try:
            os.remove(image)
        except:
            os.rmdir(image)
    os.chdir(basepath)
    

def clear_input_folder():
    os.chdir(input_path)
    file_list = os.listdir()
    for image in file_list:
        try:
            os.remove(image)
        except:
            os.rmdir(image)
    os.chdir(basepath)

In [7]:

import anvil.server

anvil.server.connect("NDXLEIAYPNCKPZZZJ67YHJ3C-4DOOCAYYMZN2YEXK")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [8]:
import anvil.media
import io
import random
from io import BytesIO
from PIL import Image


@anvil.server.callable
def upload_image(file):
    try:
        os.chdir(input_path)
        picture = Image.open(io.BytesIO(file.get_bytes()))
        picture = picture.convert("RGB")
        picture = picture.save('image.jpg', 'JPEG')
        os.chdir(basepath)
        return 'File Uploaded.'
    except:
        clear_input_folder()
        return 'File Type Not Supported, Please try a different file.'

    
@anvil.server.callable
def url_image(source_url):
    try:
        os.chdir(input_path)
        img_data = requests.get(source_url).content
        with open('image.jpg', 'wb') as handler:
            handler.write(img_data)
        return 'URL Image Loaded.'
    except:
        clear_input_folder()
        return 'URL Image Not Supported, Please try a different URL.'
    os.chdir(basepath)



@anvil.server.callable
def image_colorise(r_factor):
    os.chdir(input_path)
    images = os.listdir()
    os.chdir(output_path)
    for image in images:
        p = image_input + 'image.jpg'
        os.chdir(basepath+'\\DeOldify')
        image_path = colorizer.plot_transformed_image(path=p, render_factor=r_factor, 
                                                      compare=True, watermarked=False)
        picture = Image.open(image_path)
        os.chdir(output_path)
        picture = picture.save('COLORIZED_' + image)
        colored = Image.open('COLORIZED_' + image)
        byte_io = BytesIO()
        colored.save(byte_io, 'JPEG')
        os.chdir(basepath)
        clear_input_folder()
        clear_output_folder()
        return anvil.BlobMedia("image/jpeg", byte_io.getvalue() , name='Colorized_Image' + 
                               str(random.randint(0,9999)) + '.jpg')
    
    

In [ ]:
anvil.server.wait_forever()